In [1]:
import pandas as pd
import math

**Cleaning data**

In [2]:
dataset = pd.read_csv('Coup_Data_v2.0.0.csv', encoding='utf-8', delimiter=',', error_bad_lines=False) 

In [3]:
# убираем уже не существующие страны
clean_data = dataset[dataset.country != 'USSR']

In [4]:
# учёт разных названий одной и той же страны
clean_data = clean_data.replace(to_replace={'Kyrgyz Republic':'Kyrgyzstan', "Cote d'Ivoire":'Ivory Coast','Congo':'Democratic Republic of the Congo'})
#clean_data.iloc[279] проверить что в конкретной строке

In [5]:
# запись фрейма в файл
clean_data.to_csv('Coup_Data_v2.0.0_clean.csv', encoding='utf-8')

**Computing Coups for each country**

In [6]:
data = pd.read_csv('Coup_Data_v2.0.0_clean.csv', encoding='utf-8', delimiter=',', error_bad_lines=False) 

In [7]:
# госперевороты для заданного периода
start_date = 1989
data_filtered = data[data['year'] > start_date]

In [8]:
# сортировка стран по алфавиту
data_country_filtered = data_filtered.sort_values('country', ascending=True)
# список всех стран в алфавитном порядке
country_list = data_country_filtered['country'].unique()
# количество стран в датасете
country_num = len(country_list)
print(country_num)

83


In [9]:
# госперевороты для заданной страны
len(data[data['country'] == 'Russia'])

2

In [10]:
# частотность переворотов за последние 50 лет
# количество переворотов == количество строчек с этой страной
data_country_numbered = data_filtered.groupby('country').count()
coup_numbers = list(data_country_numbered['coup_id'])

In [11]:
# таблица для записи итоговых данных
result_data = pd.DataFrame({'country':country_list, 'coup_num':coup_numbers}, columns=['country', 'coup_num'])

In [12]:
# статистика по столбцу
result_data['coup_num'].describe()
# среднее значение
result_data['coup_num'].mean()

3.0963855421686746

In [13]:
# измение имени столбца
result_data.rename(columns={'Unnamed: 0':'id'}, inplace = True)

In [14]:
result_data.columns

Index(['country', 'coup_num'], dtype='object')

**Computing Probabilities**

**Вычисление частот и сглаженных частот**

пусть средняя частота переворотов для всех стран - m
пусть коэффициент сглаживания a « 1 (например, 0.1)

путь частота переворотов для конкретной страны mZ

p_smooth = (1-a)mZ + am

In [15]:
a = 0.1
p_month = list()
p_year = list()
p_year_smooth = list()
p_month_smooth = list()

In [16]:
# частоты по месяцам
for num in result_data['coup_num']:
   p_month.append(num/600*100) 
# частоты по годам
for num in result_data['coup_num']:
   p_year.append(num/50*100) 

In [17]:
# запись значение в фрейм
result_data['p_month'] = p_month
result_data['p_year'] = p_year

In [18]:
# сглаженные частоты по месяцам
m = result_data['p_month'].mean()
for p in result_data['p_month']:
   p_month_smooth.append(math.ceil((1-a)*p + a*m)) 
# сглаженные частоты по годам
m = result_data['p_year'].mean()
for p in result_data['p_year']:
   p_year_smooth.append(math.ceil((1-a)*p + a*m))

In [19]:
# запись значение в фрейм
result_data['p_year_smooth'] = p_year_smooth
result_data['p_month_smooth'] = p_month_smooth

In [20]:
# чтение списка стран из файла
# !!!! change path !!!
countries = pd.read_csv('/home/anastasia/PycharmProjects/coup-prediction/data/country_names.csv', encoding='utf-8', delimiter=',', error_bad_lines=False) 

In [21]:
result_data_with_un = result_data.copy()
# добавляем в список стран, которых не было в Coup d'etait
list_for_adding = list(set(countries['country_name']) - set(result_data['country']))        
# частоты для них проставляем средние
p_month_smooth_average = math.ceil(result_data['p_month_smooth'].mean())
p_month_average = math.ceil(result_data['p_month'].mean())
p_year_smooth_average = math.ceil(result_data['p_year_smooth'].mean())
p_year_average = math.ceil(result_data['p_year'].mean())
coup_average = math.ceil(result_data['coup_num'].mean())


In [22]:
data_un = pd.DataFrame({'country':list_for_adding, 'coup_num':coup_average, 'p_month': p_month_average,'p_year':p_year_average, 
                        'p_year_smooth':p_year_smooth_average, 'p_month_smooth': p_month_smooth_average}, 
                       columns=['country', 'coup_num', 'p_month', 'p_year', 'p_year_smooth', 'p_month_smooth'])

In [23]:
data_un

,country,coup_num,p_month,p_year,p_year_smooth,p_month_smooth
0,Brunei,4,1,7,7,2
1,Uruguay,4,1,7,7,2
2,Cyprus,4,1,7,7,2
3,Canada,4,1,7,7,2
4,Cameroon,4,1,7,7,2
...,...,...,...,...,...,...
109,Colombia,4,1,7,7,2
110,Saudi Arabia,4,1,7,7,2
111,South Africa,4,1,7,7,2
112,Namibia,4,1,7,7,2


In [24]:
result_data_with_un = result_data_with_un.append(data_un, ignore_index=True)

In [25]:
result_data_with_un

,country,coup_num,p_month,p_year,p_year_smooth,p_month_smooth
0,Afghanistan,4,0.666667,8.0,8,1
1,Albania,1,0.166667,2.0,3,1
2,Algeria,5,0.833333,10.0,10,1
3,Argentina,1,0.166667,2.0,3,1
4,Armenia,1,0.166667,2.0,3,1
...,...,...,...,...,...,...
192,Colombia,4,1.000000,7.0,7,2
193,Saudi Arabia,4,1.000000,7.0,7,2
194,South Africa,4,1.000000,7.0,7,2
195,Namibia,4,1.000000,7.0,7,2


In [26]:
# запись фрейма в файл
result_data_with_un.to_csv('probas_for_coup_detait_with_un.csv', encoding='utf-8')